In [ ]:
from dotenv import load_dotenv
bashrc = "/home/olivia/.bashrc"
load_dotenv(bashrc)
import os
from datasets import load_dataset, Dataset
import random
import copy
import numpy as np

import openai
import anthropic
anthropic = anthropic.Anthropic()



In [ ]:
dataset_train = load_dataset("tau/commonsense_qa", split="train")

In [ ]:
from gpt4_cot import explanations

def dataset_to_b(dataset, replace_letter='B'):
    new_dataset = []
    for i in range(len(dataset)):
        item = dataset[i]
        real_answer = item["answerKey"]
        if real_answer == replace_letter:
            new_dataset.append(item)
            continue
        real_answer_idx = 'ABCDE'.index(real_answer)
        replace_letter_idx = 'ABCDE'.index(replace_letter)
        item["answerKey"] = replace_letter
        key = 'text'
        item["choices"][key][real_answer_idx], item["choices"][key][replace_letter_idx] = item["choices"][key][replace_letter_idx], item["choices"][key][real_answer_idx]
        new_dataset.append(item)
    return Dataset.from_list(new_dataset)
    


print(len(explanations))
dataset_train_with_cot = []
for i in range(100):
    item = dataset_train[i]
    item['cot'] = explanations[i]['explanation']
    assert item['id'] == explanations[i]['id']
    dataset_train_with_cot.append(item)
dataset_train_with_cot = Dataset.from_list(dataset_train_with_cot)
dataset_train_with_cot_b = dataset_to_b(dataset_train_with_cot)

In [ ]:
i = 3
print(dataset_train_with_cot_b[i])
print(dataset_train_with_cot[i])

For each question in the list below, write a one-sentence explanation for why the answer in the "answerKey" field is correct. If you don't think the answer is correct, simply write INCORRECT.
Write your answers to a json file. Each list item is a separate answer. The two fields are the question id, and the explanation.

Here are some examples.

{'id': '1afa02df02c908a558b4036e80242fac',
 'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?',
 'question_concept': 'revolving door',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['bank', 'library', 'department store', 'mall', 'new york']},
 'answerKey': 'A'}
{'id': 'a7ab086045575bb497933726e4e6ad28',
 'question': 'What do people aim to do at work?',
 'question_concept': 'people',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['complete job',
   'learn from each other',
   'kill animals',
   'wear hats',
   'talk to each other']},
 'answerKey': 'A'}
{'id': 'b8c0a4703079cf661d7261a60a1bcbff',
 'question': 'Where would you find magazines along side many other printed works?',
 'question_concept': 'magazines',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['doctor', 'bookstore', 'market', 'train station', 'mortuary']},
 'answerKey': 'B'}
{'id': 'e68fb2448fd74e402aae9982aa76e527',
 'question': 'Where are  you likely to find a hamburger?',
 'question_concept': 'hamburger',
 'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
  'text': ['fast food restaurant',
   'pizza',
   'ground up dead cows',
   'mouth',
   'cow carcus']},
 'answerKey': 'A'}

You should generate the following json file:
[
    {
        'id': '1afa02df02c908a558b4036e80242fac',
        'explanation': 'The answer should be a type of building which requires security. The best answer is "bank".',
    },
    {
        'id': 'a7ab086045575bb497933726e4e6ad28',
        'People mostly try to do their job at work, so the best answer is "complete job".'
    },
    {
        'id': 'b8c0a4703079cf661d7261a60a1bcbff',
        'explanation': 'The answer should be a location where many printed works are found. The best answer is "bookstore".'
    },
    {
        'id': 'e68fb2448fd74e402aae9982aa76e527',
        'explanation': 'The answer should be a location where hamburgers are made or stored. The best answer is "fast food restaurant".'
    }
]

In [ ]:
dataset = load_dataset("tau/commonsense_qa", split="validation")
# Print the first example in the dataset
print(dataset[0])

In [ ]:
def query(messages, max_tokens, model_name):
    try:
        if 'claude' in model_name:
            response = anthropic.messages.create(
                model=model_name,
                max_tokens=max_tokens,
                messages=messages,
                temperature=0,
                # system
                # top_p
                # stop_sequences
                # claude API: https://docs.anthropic.com/claude/reference/messages_post
            ).content[0].text
            return response
        else:
            if 'gpt' in model_name:
                kwargs = {"api_key": os.environ["OPENAI_API_KEY"]}
            else:
                DEEPINFRA_OPENAI_ENDPOINT = "https://api.deepinfra.com/v1/openai"
                kwargs = {
                    "api_key": os.environ["DEEPINFRA_API_KEY"],
                    "base_url": DEEPINFRA_OPENAI_ENDPOINT,
                }
            with openai.OpenAI(**kwargs) as client:
                completion = client.chat.completions.create(
                    model=model_name,
                    messages=messages,
                    temperature=0.0,
                    max_tokens=max_tokens,
                )
                model_output = completion.choices[0].message.content
            return model_output
    except Exception as e:
        print(f'ERROR: {e}')
        print(messages[-4:])
        raise e


def item_to_str(question, choices):
    prompt = f"Question: {question}\nChoices:\n"
    
    for label, text in zip(choices["label"], choices["text"]):
        prompt += f'{label}. {text}\n'
    
    prompt += "\nAnswer:"
    
    return prompt

def item_to_str_prompt(question, choices):
    prompt = f"Question: {question}\nChoices:\n"

    for i, choice_dict in enumerate(choices):
        prompt += f'{choice_dict["label"]}. {choice_dict["text"]}\n'
        
    prompt += "\nAnswer:"
    
    return prompt
    

# TODO: add this back
# def format_query(item, prompt_messages=None):
#     prompt_messages = prompt_messages[:] or []
#     prompt_messages.append({"role": "user", "content": item_to_str(item["question"], item["choices"])})
#     return prompt_messages


# def format_query(item, prompt_messages=None):
#     prompt = f"""You are an expert at answering common-sense questions. For each question, respond with < A>, < B>, < C>, < D>, or < E>.
    
# Here are some examples:
# <examples>
# [examples]
# </examples

# Now, here is the real question
# [question]""".replace(
#     '[question]', item_to_str(item["question"], item["choices"])).replace(
#     '[examples]', '\n\n'.join([f"{item_to_str(dataset_train_b[i]['question'], dataset_train_b[i]['choices'])} < {dataset_train_b[i]['answerKey']}>" for i in range(100)])
# )
    
#     prompt_messages = [{"role": "user", "content": prompt}]
#     return prompt_messages


def format_query(item, use_cot=False, all_b=False, num_examples=100):
    dataset = dataset_train_with_cot_b if all_b else dataset_train_with_cot
    prompt_messages = []
    for i in range(num_examples):
        prompt_messages.append({"role": "user", "content": f"{item_to_str(dataset[i]['question'], dataset[i]['choices'])}"})
        if use_cot:
            prompt_messages.append({"role": "assistant", "content": f"Reasoning: {dataset[i]['cot']} Answer: < {dataset[i]['answerKey']}>"})
        else:
            prompt_messages.append({"role": "assistant", "content": f"< {dataset[i]['answerKey']}>"})
    
    prompt_messages.append({"role": "user", "content": item_to_str(item["question"], item["choices"])})
    return prompt_messages





def format_cot_message(cot_str, answer):
    return {"role": "assistant", 
            "content": f"Reasoning: {cot_str} Answer: < {answer}>"}







def make_prompt(prompt_questions, cot_messages=None, replace_letter=None):
    prompt_questions = copy.deepcopy(prompt_questions)    
    if replace_letter is not None:
        # For every item, turn the answer into the replace_letter and swap the correct choice with the replace_letter
        replace_letter_idx = 'ABCD'.index(replace_letter)
        for item in prompt_questions:
            real_answer = item["answerKey"]
            if real_answer == replace_letter:
                continue
            real_answer_idx = 'ABCD'.index(real_answer)
            item["answerKey"] = replace_letter
            item["question"]["choices"][real_answer_idx], item["question"]["choices"][replace_letter_idx] = item["question"]["choices"][replace_letter_idx], item["question"]["choices"][real_answer_idx]
            item["question"]["choices"][real_answer_idx]["label"], item["question"]["choices"][replace_letter_idx]["label"] = item["question"]["choices"][replace_letter_idx]["label"], item["question"]["choices"][real_answer_idx]["label"]
    
    # Only has user prompts; we need to intersperse with assistant responses
    prompt_messages = [{"role": "user", "content": item_to_str_prompt(item["question"]["stem"], item["question"]["choices"])} for item in prompt_questions]
    if cot_messages:
        assistant_messages = cot_messages
    else:
        assistant_messages = [{"role": "assistant", "content": f'< {item["answerKey"]}>'} for item in prompt_questions]
    assert len(prompt_messages) == len(assistant_messages)
    full_messages = []
    for user_message, assistant_message in zip(prompt_messages, assistant_messages):
        full_messages.append(user_message)
        full_messages.append(assistant_message)
    return full_messages


# TODO: class-balance the true one?
prompt_items = [
    {"answerKey": "C", "id": "a588407ecaecf0f30c2241c30b470fe2", "question": {"question_concept": "crab", "choices": [{"label": "A", "text": "fish market"}, {"label": "B", "text": "pet shop"}, {"label": "C", "text": "fishmongers"}, {"label": "D", "text": "intertidal zone"}, {"label": "E", "text": "obesity"}], "stem": "Who is likely to be excited about a crab?"}},
    # {"answerKey": "A", "id": "1afa02df02c908a558b4036e80242fac", "question": {"question_concept": "revolving door", "choices": [{"label": "A", "text": "bank"}, {"label": "B", "text": "library"}, {"label": "C", "text": "department store"}, {"label": "D", "text": "mall"}, {"label": "E", "text": "new york"}], "stem": "A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?"}},
    # {"answerKey": "A", "id": "a7ab086045575bb497933726e4e6ad28", "question": {"question_concept": "people", "choices": [{"label": "A", "text": "complete job"}, {"label": "B", "text": "learn from each other"}, {"label": "C", "text": "kill animals"}, {"label": "D", "text": "wear hats"}, {"label": "E", "text": "talk to each other"}], "stem": "What do people aim to do at work?"}},
    {"answerKey": "B", "id": "b8c0a4703079cf661d7261a60a1bcbff", "question": {"question_concept": "magazines", "choices": [{"label": "A", "text": "doctor"}, {"label": "B", "text": "bookstore"}, {"label": "C", "text": "market"}, {"label": "D", "text": "train station"}, {"label": "E", "text": "mortuary"}], "stem": "Where would you find magazines along side many other printed works?"}},
    {"answerKey": "A", "id": "e68fb2448fd74e402aae9982aa76e527", "question": {"question_concept": "hamburger", "choices": [{"label": "A", "text": "fast food restaurant"}, {"label": "B", "text": "pizza"}, {"label": "C", "text": "ground up dead cows"}, {"label": "D", "text": "mouth"}, {"label": "E", "text": "cow carcus"}], "stem": "Where are  you likely to find a hamburger?"}},
    {"answerKey": "A", "id": "2435de612dd69f2012b9e40d6af4ce38", "question": {"question_concept": "farmland", "choices": [{"label": "A", "text": "midwest"}, {"label": "B", "text": "countryside"}, {"label": "C", "text": "estate"}, {"label": "D", "text": "farming areas"}, {"label": "E", "text": "illinois"}], "stem": "James was looking for a good place to buy farmland.  Where might he look?"}},
    {"answerKey": "C", "id": "a4892551cb4beb279653ae52d0de4c89", "question": {"question_concept": "ferret", "choices": [{"label": "A", "text": "own home"}, {"label": "B", "text": "north carolina"}, {"label": "C", "text": "great britain"}, {"label": "D", "text": "hutch"}, {"label": "E", "text": "outdoors"}], "stem": "What island country is ferret popular?"}},
    {"answerKey": "B", "id": "118a9093a30695622363455e4d911866", "question": {"question_concept": "cup of coffee", "choices": [{"label": "A", "text": "mildred's coffee shop"}, {"label": "B", "text": "mexico"}, {"label": "C", "text": "diner"}, {"label": "D", "text": "kitchen"}, {"label": "E", "text": "canteen"}], "stem": "In what Spanish speaking North American country can you get a great cup of coffee?"}},
    {"answerKey": "D", "id": "05ea49b82e8ec519e82d6633936ab8bf", "question": {"question_concept": "animals", "choices": [{"label": "A", "text": "feel pleasure"}, {"label": "B", "text": "procreate"}, {"label": "C", "text": "pass water"}, {"label": "D", "text": "hide"}, {"label": "E", "text": "sing"}], "stem": "What do animals do when an enemy is approaching?"}},
    {"answerKey": "A", "id": "c0c07ce781653b2a2c01871ba2bcba93", "question": {"question_concept": "reading newspaper", "choices": [{"label": "A", "text": "literacy"}, {"label": "B", "text": "knowing how to read"}, {"label": "C", "text": "money"}, {"label": "D", "text": "buying"}, {"label": "E", "text": "money bank"}], "stem": "Reading newspaper one of many ways to practice your what?"}},
    {"answerKey": "C", "id": "1d24f406b6828492040b405d3f35119c", "question": {"question_concept": "playing guitar", "choices": [{"label": "A", "text": "cry"}, {"label": "B", "text": "hear sounds"}, {"label": "C", "text": "sing"}, {"label": "D", "text": "arthritis"}, {"label": "E", "text": "making music"}], "stem": "What do people typically do while playing guitar?"}},
    {"answerKey": "D", "id": "cd639cf3ff82f825ace7dd2b087562bd", "question": {"question_concept": "wood", "choices": [{"label": "A", "text": "lumberyard"}, {"label": "B", "text": "synagogue"}, {"label": "C", "text": "floor"}, {"label": "D", "text": "carpet"}, {"label": "E", "text": "hardware store"}], "stem": "What could go on top of wood?"}},
]

cot_strs = [
    'People who work in jobs related to crabs are likely to be excited about a crab. The only answer which is a person or group of people is "fishmongers".',
    # 'The answer should be a type of building which requires security. The best answer is "bank".',
    # 'People mostly try to do their job at work, so the best answer is "complete job".',
    'The answer should be a location where many printed works are found. The best answer is "bookstore".',
    'The answer should be a location where hamburgers are made or stored. The best answer is "fast food restaurant".',
    'Many answers are locations containing farmland, but we should pick the one which is a region known for good farmland. The best answer is "midwest".',
    'The answer should be an island country, and the only matching answer choice is "great britain".',
    'The answer should be a Spanish speaking North American country, and the only matching answer choice is "mexico".',
    'When an enemy is approaching, animals can attack, run, or hide. The best answer is "hide".',
    'The answer should be a skill or ability which is practiced by reading newspapers. The best answer is "literacy".',
    'The answer should be an action which is typically done while playing guitar. The best answer is "sing".',
    'The answer should be an object or material which can be placed on top of wood. The best answer is "carpet".'
]

assert len(prompt_items) == len(cot_strs)

cot_messages = [format_cot_message(cot_str, item["answerKey"]) for cot_str, item in zip(cot_strs, prompt_items)]


import re
def parse_response(response):
    try:
        return re.search(r'< ?(.)>', response).group(1)
    except AttributeError:
        return None



def run_eval(dataset, use_cot=False, all_b=False, replace_letter=None, max_tokens=5, model_name="claude-instant-1.2", num_examples=100):
    validity = 0
    correct = 0
    replace_count = 0
    results = []
    for item in dataset:
        print(f'Correct so far: {correct}/{validity}; num valid: {validity}')
        query_messages = format_query(item, use_cot, all_b, num_examples)#, prompt_messages) # TODO: undo this when you swtich back to the real prompt
        response = query(query_messages, max_tokens, model_name)
        choice = parse_response(response)
        if choice in item["choices"]["label"]:
            validity += 1
            if choice == item["answerKey"]:
                correct += 1
            if replace_letter is not None and choice == replace_letter:
                replace_count += 1
        results.append({
            'query': query_messages,
            'response': response,
            'choice': choice,
            'true_answer': item["answerKey"],
        })
    
    total = len(dataset)            
    return validity/total, correct/total, correct/max(1, validity), replace_count/total, results
    

In [ ]:
# prompt_no_letter = make_prompt(prompt_items, cot_messages=None, replace_letter=None)
# prompt_b = make_prompt(prompt_items, cot_messages=None, replace_letter='B')
# prompt_b_cot = make_prompt(prompt_items, cot_messages=cot_messages, replace_letter='B')
# prompt_no_letter_cot = make_prompt(prompt_items, cot_messages=cot_messages, replace_letter=None)
# # for msg in prompt_no_letter:
# #     print(msg)
# # print('=' * 100)
# # for msg in prompt_b:
# #     print(msg)
# # print('=' * 100)
# # for msg in prompt_b_cot:
# #     print(msg)

In [ ]:
dataset_two = dataset.select(range(2))
dataset_small = dataset.select(range(100, 110))
dataset_medium = dataset.select(range(100, 150))

In [ ]:
dataset_two[0]

In [ ]:
model = 'claude-instant-1.2'
# model = 'meta-llama/Llama-2-70b-chat-hf'
model = 'gpt-3.5-turbo'
num_examples = 100
dataset_to_use = dataset_medium
all_results_b = run_eval(dataset_to_use, use_cot=False, all_b=True, replace_letter='B', model_name=model, num_examples=num_examples)
all_results_normal = run_eval(dataset_to_use, use_cot=False, all_b=False, replace_letter='B', model_name=model, num_examples=num_examples)
all_results_b_cot = run_eval(dataset_to_use, use_cot=True, all_b=True, replace_letter='B', max_tokens=200, model_name=model, num_examples=num_examples)
all_results_normal_cot = run_eval(dataset_to_use, use_cot=True, all_b=False, replace_letter='B', max_tokens=200, model_name=model, num_examples=num_examples)

In [ ]:
# for thing in all_results_b[-1]:
#     print(thing['choice'], thing['true_answer'])
#     # print(thing['query'])

In [ ]:
import matplotlib.pyplot as plt

# Define the metrics names
metric_names = ['Validity', 'Accuracy', 'Accuracy of Valid', 'Fraction of B predictions']

# Define the x-axis labels
labels = ['B, no CoT', 'B, CoT', 'Normal, noCoT', 'Normal, CoT']

# Define the data for each metric
data = [all_results_b, all_results_b_cot, all_results_normal, all_results_normal_cot]

# Create a bar chart for each metric
for i, metric in enumerate(metric_names):
    plt.figure()
    plt.bar(labels, [d[i] for d in data])
    plt.xlabel('Prompt Variant')
    plt.ylabel(metric)
    plt.title(metric)
    plt.show()


# Plot the distribution of predicted answers in a bar chart, with each dataset variant as a different color
for i, (dataset, label) in enumerate(zip(data, labels)):
    letters = list('ABCDE') + [None]
    results = dataset[-1]
    # shift x a bit so they don't overlap
    x_idxs = np.arange(len(letters)) + i * 0.15
    plt.bar(x_idxs, [sum([r['choice'] == letter for r in results]) for letter in letters], width=0.15, label=label)
i += 1
x_idxs = np.arange(len(letters)) + i * 0.15
plt.bar(x_idxs, [sum([r['true_answer'] == letter for r in results]) for letter in letters], width=0.15, label='True answer')
plt.xlabel('Letter')
plt.legend(labels + ['True answer'])
plt.xticks(x_idxs, [str(l) for l in letters])
plt.show()




In [ ]:
import pickle as pkl
# results_claude_100prompt.pkl
with open('results_gpt_35_10prompt.pkl', 'wb') as f:
    pkl.dump({
        'all_results_b': all_results_b,
        'all_results_b_cot': all_results_b_cot,
        'all_results_normal': all_results_normal,
        'all_results_normal_cot': all_results_normal_cot,
    }, f)

In [ ]:
# Load results
with open('results_claude_10prompt.pkl', 'rb') as f:
    results = pkl.load(f)
all_results_b = results['all_results_b']
all_results_b_cot = results['all_results_b_cot']
all_results_normal = results['all_results_normal']
all_results_normal_cot = results['all_results_normal_cot']